In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

# SUSB

## Build Croswalks

Following G&T, we create a weighted mapping from each standard to the next based on the concordance of 6-digit industries.

In [2]:
def get_weights(concord, fromstd, tostd, digits):
    """
    Given a concordance between two NAICS standards, calculate the weights
    for converting from the fromstd to the tostd based on the number of
    matched rows, each of which represents a 6-digit industry.
    """
    return (
        concord
        [[f'{fromstd}_{digits}digit', f'{tostd}_{digits}digit']]
        .assign(weight=lambda df: (
            # Assign weights of 1 / # of 6-digit inds in from standard
            1 / (df[f'{fromstd}_{digits}digit'].map(
                df.groupby(f'{fromstd}_{digits}digit')
                .count()[f'{tostd}_{digits}digit']
            ))
        ))
        #sum by pair to generate final weights
        .groupby([f'{fromstd}_{digits}digit', f'{tostd}_{digits}digit'])
        .sum()
        .reset_index()
        .rename(columns=lambda x: x.split('_')[0])
    )

In [3]:
concord9702 = (
    pd.read_excel(
        'data/xwalks/1997_NAICS_to_2002_NAICS.xls',
        sheet_name=1,
        skipfooter=1,
        usecols=[0, 2],
        names=['naics97_6digit', 'naics02_6digit']
    )
    .assign(naics97_4digit=lambda df: df['naics97_6digit'] // 100)
    .assign(naics97_3digit=lambda df: df['naics97_6digit'] // 1000)
    .assign(naics02_4digit=lambda df: df['naics02_6digit'] // 100)
    .assign(naics02_3digit=lambda df: df['naics02_6digit'] // 1000)
)
concord9702.head()

,naics97_6digit,naics02_6digit,naics97_4digit,naics97_3digit,naics02_4digit,naics02_3digit
0,111110,111110,1111,111,1111,111
1,111120,111120,1111,111,1111,111
2,111130,111130,1111,111,1111,111
3,111140,111140,1111,111,1111,111
4,111150,111150,1111,111,1111,111


In [4]:
concord0207 = (
    pd.read_excel(
        'data/xwalks/2002_to_2007_NAICS.xls',
        skiprows=2,
        usecols=[0, 2],
        names=['naics02_6digit', 'naics07_6digit']
    )
    .assign(naics02_4digit=lambda df: df['naics02_6digit'] // 100)
    .assign(naics02_3digit=lambda df: df['naics02_6digit'] // 1000)
    .assign(naics07_4digit=lambda df: df['naics07_6digit'] // 100)
    .assign(naics07_3digit=lambda df: df['naics07_6digit'] // 1000)
)
concord0207.head()

,naics02_6digit,naics07_6digit,naics02_4digit,naics02_3digit,naics07_4digit,naics07_3digit
0,111110,111110,1111,111,1111,111
1,111120,111120,1111,111,1111,111
2,111130,111130,1111,111,1111,111
3,111140,111140,1111,111,1111,111
4,111150,111150,1111,111,1111,111


In [5]:
concord1207 = (
    pd.read_excel(
        'data/xwalks/2007_to_2012_NAICS.xls',
        skiprows=2,
        usecols=[0, 2],
        names=['naics07_6digit', 'naics12_6digit']
    )
    .assign(naics12_4digit=lambda df: df['naics12_6digit'] // 100)
    .assign(naics12_3digit=lambda df: df['naics12_6digit'] // 1000)
    .assign(naics07_4digit=lambda df: df['naics07_6digit'] // 100)
    .assign(naics07_3digit=lambda df: df['naics07_6digit'] // 1000)
)
concord1207.head()

,naics07_6digit,naics12_6digit,naics12_4digit,naics12_3digit,naics07_4digit,naics07_3digit
0,111110,111110,1111,111,1111,111
1,111120,111120,1111,111,1111,111
2,111130,111130,1111,111,1111,111
3,111140,111140,1111,111,1111,111
4,111150,111150,1111,111,1111,111


In [6]:
xwalk9702 = pd.concat(tuple(
    get_weights(concord9702, 'naics97', 'naics02', digits)
    for digits in (3, 4)
))
xwalk9702.head()

,naics97,naics02,weight
0,111,111,1.0
1,112,112,1.0
2,113,113,1.0
3,114,114,1.0
4,115,115,1.0


In [7]:
xwalk0207 = pd.concat(tuple(
    get_weights(concord0207, 'naics02', 'naics07', digits)
    for digits in (3, 4)
))
xwalk0207.head()

,naics02,naics07,weight
0,111,111,0.96875
1,111,112,0.03125
2,112,112,1.00000
3,113,113,1.00000
4,114,114,1.00000


In [8]:
# NB: Converting from NAICS 2012 to 2007, not vice versa
xwalk1207 = pd.concat(tuple(
    get_weights(concord1207, 'naics12', 'naics07', digits)
    for digits in (3, 4)
))
xwalk1207.head()

,naics12,naics07,weight
0,111,111,1.0
1,112,112,1.0
2,113,113,1.0
3,114,114,1.0
4,115,115,1.0


## Convert and Compile

We read in 98/99 through 06/07 from the SUSB Table files, which all follow the same format.
We read in 07/08 through 15/16 from the SUSB Database files.

The NAICS standards used are:

start year | end year | standard
:-:|:-:|:-:
98/99 | 02/03 | NAICS 97
03/04 | 07/08 | NAICS 02
08/09 | 12/13 | NAICS 07
13/14 | 15/16 | NAICS 12

All SUSB vintages are converted to NAICS 2007


In [9]:
def convert_naics(raw, fromstd, tostd, xwalk):
    """
    Given data in one standard, convert to another given a crosswalk
    """
    return (
        raw
        .rename(columns={'naics_code': fromstd})
        .merge(xwalk.rename(columns={tostd: 'naics_code'}), how='outer')
        .assign(value = lambda df: df['value'].mul(df['weight']))
        .groupby(['year', 'naics_code', 'var'])[['value']]
        .sum(min_count=1)
        .reset_index()
    )

In [10]:
VARMAP_TABLE = {
    1: 'estb_initial',
    4: 'estb_births',
    5: 'estb_deaths',
    6: 'estb_expand',
    7: 'estb_contract',
    10: 'empl_initial',
    13: 'empl_births',
    14: 'empl_deaths',
    15: 'empl_expand',
    16: 'empl_contract',    
}

def read_susb_table(initial_year):
    """Read in SUSB table of the format used from the 98-99 to 06-07 vintages"""
    print(initial_year)
    return (
        pd.read_excel(
            f'data/susb/us_4digitnaics_emplchange_{initial_year}-{initial_year+1}.xls',
            skiprows=7,
            na_values=['(D)', '[j]', '[k]', '[i]'],
        )
        .rename(columns={
            'CODE': 'naics_code',
            'CODE.1': 'var_code',
            'TOTAL': 'value',
        })
        .loc[lambda df: df['naics_code'].astype(str).str.contains('\d+')]
        .assign(naics_code=lambda df: (
            df['naics_code']
            .astype(str)
            .str.strip('-')
            .str.split('-').str[0]
            .astype(int)
        ))
        .loc[lambda df: df['naics_code'] >= 100]
        .loc[lambda df: df['var_code'].isin(VARMAP_TABLE)]
        .assign(var=lambda df: df['var_code'].map(VARMAP_TABLE))
        .assign(year=initial_year)
        [['year', 'naics_code', 'var', 'value']]
    )

In [11]:
VARMAP_DB = {
    'NAICS': 'naics_code',
    'INIT_ESTB': 'estb_initial',
    'INIT_EMPL': 'empl_initial',
    'INIT_EMPLFL_N': 'empl_initial_flag',
    'BIRTHS_ESTB': 'estb_births',
    'BIRTHS_EMPL': 'empl_births',
    'BIRTHS_EMPLFL_N': 'empl_births_flag',
    'DEATHS_ESTB': 'estb_deaths',
    'DEATHS_EMPL': 'empl_deaths',
    'DEATHS_EMPLFL_N': 'empl_deaths_flag',
    'EXP_ESTB': 'estb_expand',
    'EXP_EMPL': 'empl_expand',
    'EXP_EMPLFL_N': 'empl_expand_flag',
    'CONTR_ESTB': 'estb_contract',
    'CONTR_EMPL': 'empl_contract',
    'CONTR_EMPLFL_N': 'empl_contract_flag'
}
def read_susb_db(initial_year):
    """Read in SUSB databases as used from the 2007-2008 vintage on"""
    print(initial_year)
    return (
        pd.read_csv(f'data/susb/us_state_emplchange_{initial_year}-{initial_year + 1}.txt')
        .loc[lambda df: df['STATE'] == 0]
        .loc[lambda df: df['ENTRSIZE'] == 1]
        [VARMAP_DB.keys()]
        .rename(columns=VARMAP_DB)
        .loc[lambda df: df['naics_code'].astype(str).str.contains('\d+')]
        .assign(naics_code=lambda df: (
            df['naics_code']
            .astype(str)
            .str.strip('-')
            .str.split('-').str[0]
            .astype(int)
        ))
        .loc[lambda df: df['naics_code'] >= 100]
        .assign(empl_initial=lambda df: (
            df['empl_initial']
            [~df['empl_initial_flag'].isin({'D', 'S'})] # Drop Missing or Suppressed
        ))
        .assign(empl_births=lambda df: (
            df['empl_births']
            [~df['empl_births_flag'].isin({'D', 'S'})]
        ))
        .assign(empl_deaths=lambda df: (
            df['empl_deaths']
            [~df['empl_deaths_flag'].isin({'D', 'S'})]
        ))
        .assign(empl_expand=lambda df: (
            df['empl_expand']
            [~df['empl_expand_flag'].isin({'D', 'S'})]
        ))
        .assign(empl_contract=lambda df: (
            df['empl_contract']
            [~df['empl_contract_flag'].isin({'D', 'S'})]
        ))
        .drop([i for i in VARMAP_DB.values() if i.endswith('_flag')], axis='columns')
        .melt(id_vars=['naics_code'], var_name='var')
        .assign(year=initial_year)
        [['year', 'naics_code', 'var', 'value']]
    )

Compile all years using 1997 NAICS

In [12]:
susb_naics97 = pd.concat(tuple(
    read_susb_table(initial_year)
    for initial_year in range(1998, 2003)
))
susb_naics97.sort_values(['year', 'naics_code', 'var']).head()

1998
1999
2000
2001
2002


,year,naics_code,var,value
52,1998,113,empl_births,5619.0
55,1998,113,empl_contract,-10608.0
53,1998,113,empl_deaths,-6013.0
54,1998,113,empl_expand,10818.0
49,1998,113,empl_initial,84150.0


Convert naics97 data to naics02 and read in naics02 years of SUSB. Compile all into one new dataset

In [13]:
susb_naics02 = [convert_naics(susb_naics97, 'naics97', 'naics02', xwalk9702)]
susb_naics02.extend([
    read_susb_table(initial_year)
    for initial_year in range(2003, 2007)
])
susb_naics02.append(read_susb_db(2007))
susb_naics02 = pd.concat(susb_naics02)
susb_naics02.sort_values(['year', 'naics_code', 'var']).head()

2003
2004
2005
2006
2007


,year,naics_code,var,value
0,1998.0,113,empl_births,5619.0
1,1998.0,113,empl_contract,-10608.0
2,1998.0,113,empl_deaths,-6013.0
3,1998.0,113,empl_expand,10818.0
4,1998.0,113,empl_initial,84150.0


Read in and compile naics12 years

In [14]:
susb_naics12 = pd.concat([
    read_susb_db(initial_year)
    for initial_year in range(2013, 2015)
])

2013
2014


Create the naics07 dataset for analysis by converting the naics02 and naics12 datasets to naics07, reading in the naics07 SUSB files, and compiling together

In [15]:
susb_naics07 = [
    convert_naics(susb_naics02, 'naics02', 'naics07', xwalk0207)
]
susb_naics07.extend([
    read_susb_db(initial_year)
    for initial_year in range(2008, 2012)
])
susb_naics07.append(
    convert_naics(susb_naics12, 'naics12', 'naics07', xwalk1207)
)
susb_naics07 = pd.concat(susb_naics07)
susb_naics07.head()

2008
2009
2010
2011


,year,naics_code,var,value
0,1998.0,113,empl_births,5619.0
1,1998.0,113,empl_contract,-10608.0
2,1998.0,113,empl_deaths,-6013.0
3,1998.0,113,empl_expand,10818.0
4,1998.0,113,empl_initial,84150.0


## Calculate GT Variables

In [16]:
susb = (
    susb_naics07
    .set_index(['year', 'naics_code', 'var'])
    ['value']
    .unstack()
    .reset_index()
    .assign(empl_created=lambda df: df['empl_births'] + df['empl_expand'])
    .assign(empl_destroyed=lambda df: 0 - df['empl_deaths'] - df['empl_contract'])
)

dhs = pd.DataFrame({
    'year': susb['year'] + 1,
    'naics_code': susb['naics_code'],
    'estb_dhs': susb['estb_initial'] + (susb['estb_births'] - susb['estb_deaths']) / 2,
    'empl_dhs': susb['empl_initial'] + (susb['empl_created'] - susb['empl_destroyed']) / 2,
})

susb = (
    susb
    .merge(dhs, how='outer')
    .assign(estb_birth_rate=lambda df: 100 * df['estb_births'] / df['estb_dhs'])
    .assign(empl_create_rate=lambda df: 100 * df['empl_created'] / df['empl_dhs'])
    .assign(empl_destroy_rate=lambda df: 100 * df['empl_destroyed'] / df['empl_dhs'])
)
susb.head()

,year,naics_code,empl_births,empl_contract,empl_deaths,empl_expand,empl_initial,estb_births,estb_contract,estb_deaths,estb_expand,estb_initial,empl_created,empl_destroyed,estb_dhs,empl_dhs,estb_birth_rate,empl_create_rate,empl_destroy_rate
0,1998.0,113,5619.0,-10608.0,-6013.0,10818.0,84150.0,1446.0,3495.0,1647.0,3512.0,12255.0,16437.0,16621.0,NaN,NaN,NaN,NaN,NaN
1,1998.0,114,1106.0,-1244.0,-785.0,1579.0,9312.0,287.0,267.0,233.0,268.0,1584.0,2685.0,2029.0,NaN,NaN,NaN,NaN,NaN
2,1998.0,115,8096.0,-11921.0,-6773.0,14643.0,93588.0,1190.0,2104.0,1095.0,2329.0,8740.0,22739.0,18694.0,NaN,NaN,NaN,NaN,NaN
3,1998.0,211,5945.0,-15757.0,-5644.0,8268.0,97037.0,564.0,2043.0,968.0,1067.0,7251.0,14213.0,21401.0,NaN,NaN,NaN,NaN,NaN
4,1998.0,212,7521.0,-22715.0,-10536.0,16639.0,225356.0,598.0,2266.0,636.0,2308.0,6792.0,24160.0,33251.0,NaN,NaN,NaN,NaN,NaN


## Save SUSB

In [17]:
susb.to_csv('data/final/susb_combined.csv', index=False)

# RegData
## RegData 2.0

In [18]:
rd20_3digit = (
    pd.read_csv('data/rd20/three_digit_industry_RegData2_0.csv')
    .groupby(['year', 'industry_code'])['industry_regulation_index'].sum()
    .reset_index()
    .rename(columns={
        'industry_code': 'naics_code',
        'industry_regulation_index': 'regdata20'
    })
    .loc[lambda df: df['regdata20'] > 0]
)
rd20_3digit.head()

,year,naics_code,regdata20
0,1997.0,111.0,1891.450882
1,1997.0,112.0,3704.735092
2,1997.0,113.0,399.963253
3,1997.0,114.0,18748.384140
4,1997.0,211.0,23234.321203


In [19]:
rd20_4digit = (
    pd.read_csv('data/rd20/four_digit_industry_RegData2_0.csv')
    .groupby(['year', 'industry_code'])['industry_regulation_index'].sum()
    .reset_index()
    .rename(columns={
        'industry_code': 'naics_code',
        'industry_regulation_index': 'regdata20'
    })
    .loc[lambda df: df['regdata20'] > 0]
)
rd20_4digit.head()

,year,naics_code,regdata20
0,1997,1111,0.012332
1,1997,1113,0.007914
2,1997,1114,0.003517
3,1997,1121,1190.340274
4,1997,1122,52.336162


## RegData 2.1

In [20]:
rd21_3digit = (
    pd.read_csv(
        'data/rd21/industry_classification_probabilities.csv',       
        index_col=['year', 'title', 'part']
    )
    .mul(
        (
            pd.read_csv(
                'data/rd21/cfrref.csv',
                index_col=['year', 'title', 'part'],
                encoding='latin-1'
            )
            ['restrictions']
        ),
        axis=0
    )
    .groupby(level='year').sum()
    .reset_index()
    .melt(id_vars='year', var_name='naics_code', value_name='regdata21')
    .assign(naics_code=lambda df: df['naics_code'].astype(int))
)
rd21_3digit.head()

,year,naics_code,regdata21
0,1975,111,5253.036888
1,1976,111,5293.855761
2,1977,111,5449.930892
3,1978,111,4639.366092
4,1979,111,4527.049841


## RegData 2.2

In [21]:
rd22_3digit = (
    pd.read_csv(
        'data/rd22/RegData_2_2_by_3-digit_industry.csv',
        names=['year', 'naics_code', 'restrictions', 'words'],
        skiprows=1,
    )
    .rename(columns={'restrictions': 'regdata22'})
    [['year', 'naics_code', 'regdata22']]
)
rd22_3digit.head()

,year,naics_code,regdata22
0,1970,111,6761.708427
1,1970,112,4318.580844
2,1970,114,986.732706
3,1970,115,1382.225681
4,1970,211,2271.600198


In [22]:
rd22_4digit = (
    pd.read_csv(
        'data/rd22/RegData_2_2_by_4-digit_industry.csv',
        names=['year', 'naics_code', 'restrictions', 'words'],
        skiprows=1
    )
    .rename(columns={'restrictions': 'regdata22'})
    [['year', 'naics_code', 'regdata22']]
)
rd22_4digit.head()

,year,naics_code,regdata22
0,1970,1112,54.242533
1,1970,1113,155.283095
2,1970,1114,141.363160
3,1970,1121,2807.359942
4,1970,1122,133.869355


## RegData 3.1

In [23]:
rd31_3digit = (
    pd.read_csv('data/rd31/cfr_naics07-3digit_extract.csv')
    .rename(columns={'industry-relevant restrictions': 'regdata31'})
    [['year', 'naics_code', 'regdata31']]
)
rd31_3digit.head()

,year,naics_code,regdata31
0,1970,111,5818.8343
1,1970,112,9063.2562
2,1970,114,757.5828
3,1970,115,438.9153
4,1970,211,1983.1176


In [24]:
rd31_4digit = (
    pd.read_csv('data/rd31/cfr_naics07-4digit_extract.csv')
    .rename(columns={'industry-relevant restrictions': 'regdata31'})
    [['year', 'naics_code', 'regdata31']]
)
rd31_4digit.head()

,year,naics_code,regdata31
0,1970,1111,4993.1109
1,1970,1112,5105.9292
2,1970,1113,4352.8394
3,1970,1114,4320.0242
4,1970,1119,7429.0775


# Final Compiled Datasets

In [25]:
combined_3digit = (
    susb[
        (susb['naics_code'] >= 100)
        & (susb['naics_code'] < 814) # Eliminate private households & public administration
    ]
    .assign(estb_births=lambda df: df[df['estb_births'] > 0]['estb_births'])
    .assign(estb_deaths=lambda df: df[df['estb_deaths'] > 0]['estb_deaths'])
    .assign(empl_created=lambda df: df[df['empl_created'] > 0]['empl_created'])
    .merge(rd20_3digit, how='left')
    .merge(rd21_3digit, how='left')
    .merge(rd22_3digit, how='left')
    .merge(rd31_3digit, how='left')
)
combined_3digit.to_csv('data/final/combined_3digit.csv', index=False)
combined_3digit.head()

,year,naics_code,empl_births,empl_contract,empl_deaths,empl_expand,empl_initial,estb_births,estb_contract,estb_deaths,...,empl_destroyed,estb_dhs,empl_dhs,estb_birth_rate,empl_create_rate,empl_destroy_rate,regdata20,regdata21,regdata22,regdata31
0,1998.0,113,5619.0,-10608.0,-6013.0,10818.0,84150.0,1446.0,3495.0,1647.0,...,16621.0,NaN,NaN,NaN,NaN,NaN,393.953985,4387.428106,NaN,NaN
1,1998.0,114,1106.0,-1244.0,-785.0,1579.0,9312.0,287.0,267.0,233.0,...,2029.0,NaN,NaN,NaN,NaN,NaN,19014.211936,16010.209662,5592.539131,3316.4159
2,1998.0,115,8096.0,-11921.0,-6773.0,14643.0,93588.0,1190.0,2104.0,1095.0,...,18694.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2145.552466,1553.9040
3,1998.0,211,5945.0,-15757.0,-5644.0,8268.0,97037.0,564.0,2043.0,968.0,...,21401.0,NaN,NaN,NaN,NaN,NaN,22298.961579,66989.810756,10369.798674,4184.1912
4,1998.0,212,7521.0,-22715.0,-10536.0,16639.0,225356.0,598.0,2266.0,636.0,...,33251.0,NaN,NaN,NaN,NaN,NaN,25860.865965,21824.120103,NaN,12604.6857


In [26]:
combined_4digit = (
    susb[
        (susb['naics_code'] >= 1000)
        & (susb['naics_code'] < 8140) # Eliminate private households & public administration
    ]
    .assign(estb_births=lambda df: df[df['estb_births'] > 0]['estb_births'])
    .assign(estb_deaths=lambda df: df[df['estb_deaths'] > 0]['estb_deaths'])
    .assign(empl_created=lambda df: df[df['empl_created'] > 0]['empl_created'])
    .merge(rd20_4digit, how='left')
    .merge(rd22_4digit, how='left')
    .merge(rd31_4digit, how='left')
)
combined_4digit.to_csv('data/final/combined_4digit.csv', index=False)
combined_4digit.head()

,year,naics_code,empl_births,empl_contract,empl_deaths,empl_expand,empl_initial,estb_births,estb_contract,estb_deaths,...,empl_created,empl_destroyed,estb_dhs,empl_dhs,estb_birth_rate,empl_create_rate,empl_destroy_rate,regdata20,regdata22,regdata31
0,1998.0,1131,NaN,-359.0,NaN,311.0,3426.0,52.0,71.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1231.974776,NaN,NaN
1,1998.0,1132,NaN,-205.0,NaN,196.0,1387.0,36.0,52.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.025343,506.852745,NaN
2,1998.0,1133,5269.0,-10044.0,-5509.0,10311.0,79337.0,1358.0,3372.0,1577.0,...,15580.0,15553.0,NaN,NaN,NaN,NaN,NaN,56.760083,497.848855,NaN
3,1998.0,1141,898.0,-1101.0,-705.0,1287.0,7205.0,228.0,215.0,202.0,...,2185.0,1806.0,NaN,NaN,NaN,NaN,NaN,15639.288594,5158.961576,2022.5002
4,1998.0,1142,208.0,-143.0,-80.0,292.0,2107.0,59.0,52.0,31.0,...,500.0,223.0,NaN,NaN,NaN,NaN,NaN,2836.641753,NaN,3348.3968
